In [ ]:
メタボの分類
データ変換
回帰
スコアの評価
データ化

関数化


In [ ]:
# ! pip3 install patsy
# ! pip3 install statsmodels
# ! pip3 install --upgrade pip
# ! pip3 install pandas
#import, def, path
import pandas as pd
import os
#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression

def make_pair(target_list, span):#リストから指定の間隔をあけて二つを取り出す
    pairs = [[target_list[i], target_list[i + span]] for i in range(len(target_list) - span)]
    return pairs
def marge_columns(df, columns):#カラムを追加する
    mid = pd.DataFrame(index=df.index, columns=columns)
    return pd.concat([df, mid], axis=1)
def check_metS(df, metS_facter):#メタボ因子を数値化
    metS_keys = list(metS_facter.keys())
    metS_values = list(metS_facter.values())
    df['check_腹囲'] = list(eval(f"df['{metS_keys[0]}'] {metS_values[0]}"))
    df['check_血圧'] = list(eval(f"df['{metS_keys[1]}'] {metS_values[1]}"))
    df['check_血糖'] = list(eval(f"df['{metS_keys[2]}'] {metS_values[2]}"))
    df['check_脂肪'] = list(eval(f"df['{metS_keys[3]}'] {metS_values[3]}"))
    return df
def categorize_facters(df, categorize_facter):# カテゴリー変数を２値化、trueが１、true_valueはリスト
    facters = list(categorize_facter.keys())
    true_values = list(categorize_facter.values())
    for i, true_value in enumerate(true_values):
        k += f"df['{facters[i]}'] == {true_value} |"
    df[f'categorize_{facters[i]}'] = list(eval(k[: -1]))
    return df

df_name = pd.read_csv('/Volumes/USBDISK/AHHDC/sort_pg/練習用保健データ.csv',  encoding = 'cp932')
#main_df_path = os.path.join(os.getcwd(), f'{df_name}.csv')
target_list = list(range(2015, 2021))
span = 4
pairs_column = '検診実施年度'
metS_facter = {'腹囲':'>= 85', '血圧':'>= 130', '血糖':'>= 110', '脂肪':'>= 150'}
metS_keys = list(metS_facter.keys())

In [ ]:
df = df_name.copy()
#df = df['空腹時血糖']

In [ ]:
import numpy as np
df = df_name.copy()
metS_facter = [
    ['性別', '腹囲'],               # 腹囲
    ['収縮機血圧', '拡張期血圧'],     # 血圧
    ['空腹時血糖', 'HbA1c'],        # 血糖
    ['中性脂肪', 'HDL']             # 脂質
]

df['腹囲_'] = (((df[metS_facter[0][0]] == 1) & (df[metS_facter[0][1]] >= 85) |   # 男性
              (df[metS_facter[0][0]] == 0) & (df[metS_facter[0][1]] >= 90))     # 女性
               .where((df[metS_facter[0][0]].notnull()) & (df[metS_facter[0][1]].notnull()), np.nan)) 
df['血圧_'] = (((df[metS_facter[1][0]] >= 130) |    # 収縮期血圧
              (df[metS_facter[1][1]] >= 85))       # 拡張期血圧
               .where((df[metS_facter[1][0]].notnull()) & (df[metS_facter[1][1]].notnull()), np.nan))
df['血糖_'] = (((df[metS_facter[2][0]] >= 110) |    # 空腹時血糖
              (df[metS_facter[2][1]] >= 5.6))             # HbA1c
               .where((df[metS_facter[2][0]].notnull()) & (df[metS_facter[2][1]].notnull()), np.nan))
df['脂質_'] = (((df[metS_facter[3][0]] >= 150) |    # 中性脂肪
              (df['HDL'] <= 40))               # HDL
               .where((df[metS_facter[3][0]].notnull()) & (df[metS_facter[3][1]].notnull()), np.nan))
# nanの変換
# df['腹囲'] = df['腹囲'].where((~df['腹囲'].notnull()) | (~df['性別'].notnull()), np.nan)
# df['血圧'] = df['血圧'].where((~df['収縮機血圧'].notnull()) | (~df['拡張期血圧'].notnull()), np.nan)
# df['血糖'] = df['血糖'].where((~df['空腹時血糖'].notnull()) | (~df['HbA1c'].notnull()), np.nan)
# df['脂質'] = df['脂質'].where((~df['中性脂肪'].notnull()) | (~df['HDL'].notnull()), np.nan)

# 判定
df['MetS'] = (df['腹囲_'] == True) & ((df['血圧_'] + df['血糖_'] + df['脂質_'] >= 2))

df_metS = df[df['MetS'] == True]
df_non_metS = df[df['MetS'] == False]

In [ ]:
#main
df = df_name
id_column = 'Unnamed: 0'
categorize_facter = {'namae':[1,2]}

pairs = make_pair(target_list, span, id_column, categorize_facter)
for pair in pairs:
    df1 = df.query(f'{pairs_column} == {pair[0]}')#train
    df1 = check_metS(df1, metS_facter)
    df1 = categorize_facters(df1, categorize_facter)
    df2 = df.query(f'{pairs_column} == {pair[1]}')#test
    df2 = check_metS(df2, metS_facter)
    df2 = df2.filter(regex = f'^({id_column}| check_)', axis = 1)#testは必要のないもの以外drop

In [ ]:
df = df_name
#df = marge_columns(df, ['check_腹囲', 'check_血圧', 'check_血糖', 'check_脂肪'])
metS_values = list(metS_facter.values())
a = '検診実施年度'
true_value = [2018, 2019]
k = ''
for i in true_value:
    k += f"(df['{a}'] == {i}) |"

df[f'check_{a}'] = list(eval(k[:-1]))




各群にrisk_facter_name

比べてchange
変化した：１
変化なし：０


In [ ]:

column_name = '睡眠時間'


X = df[[column_name]] # 説明変数
Y = df['species'].map({'versicolor': 0, 'virginica': 1}) # versicolorをクラス0, virginicaをクラス1とする
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0) # 80%のデータを学習データに、20%を検証データにする

lr = LogisticRegression() # ロジスティック回帰モデルのインスタンスを作成
lr.fit(X_train, Y_train) # ロジスティック回帰モデルの重みを学習


Work


In [ ]:

#main

df = pd.read_csv(main_df_path)
id_column = 'Unnamed: 0'
categorize_facter = {'namae':[1,2]}
facter_bp = '血圧'
bp_threshold = '>= 135'
columns_list = list(df.columns)
pairs = make_pair(target_list, span, id_column, categorize_facter)
for pair in pairs:
    df1 = df.query(f'{pairs_column} == {pair[0]}')#train
    df1['check_高血圧'] = list(eval(f"df1['{facter_bp}'] {bp_threshold}"))
    df1 = df1[df1['check_高血圧'] == False]     #元々正常のみ

    df2 = df.query(f'{pairs_column} == {pair[1]}')#test
    df1['check_高血圧_after'] = list(eval(f"df2['{facter_bp}'] {bp_threshold}"))
    df2 = df2.filter(regex = f'^({id_column}| check_)', axis = 1)#testは必要のないもの以外drop

    df1.join(df2, on = id_column)
    df1.dropna(subset=['check_高血圧_after'], inplace = True)
    df1['bp_delta'] = df1['check_高血圧_after'] - df1['check_高血圧']

    x = df1[['name']] # 説明変数
    y = df1['bp_delta'].map({0: 0, 1: 1}) # falseをクラス0, Trueをクラス1とする
    X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, random_state = 0) # 80%のデータを学習データに、20%を検証データにする


    lr = LogisticRegression() # ロジスティック回帰モデルのインスタンスを作成
    lr.fit(X_train, Y_train) # ロジスティック回帰モデルの重みを学習


    df_model["偏回帰係数"] = model.coef_[0]


In [ ]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm

# データの読み込み
exam_data = pd.read_csv("exam_data.csv")
# データのグラフ化


# モデル化
logistic_model = smf.glm(formula = "result ~ hours", # 目的変数~説明変数
                       data = exam_data, 
                       family=sm.families.Binomial(link=sm.genmod.families.links.logit())
                        )

logistic_result = logistic_model.fit()

logistic_result.summary()


### X2乗検定

In [ ]:
import pandas as pd
from scipy import stats
chi2, p, dof, exp = stats.chi2_contingency(df_name['検診実施年度','市町村名'], correction=False) #correction:イェーツの補正
print("期待度数", "\n", exp)
print("自由度", "\n", dof)
print("カイ二乗値", "\n", chi2)
print("p値", "\n", p)

In [ ]:
from itertools import combinations
def chi_squeretest(df, facters_list = []):
    facters_pairs = [[facters_pair] for facters_pair in combinations(facters_list, 2)]
    separate_facter = []
    for facters_pair in facters_pairs:
        chi2, p, dof, exp = stats.chi2_contingency(df[facters_pair], correction=False) #correction:イェーツの補正
        if p < 0.05 :
            separate_facter.append(facters_pair)
    return separate_facter#[[a,a],[b,b]]


In [ ]:
df_name

In [ ]:
import pandas as pd
from scipy import stats
facters_list = ['検診実施年度', '市町村名']
categorize_threshhold = 3
df = df_name
#X2検定
from itertools import combinations
def chi_squeretest(df, categorize_threshhold = 3, facters_list = []):
    categorize_facters_list = [facter for facter in facters_list if len(df[facter].unique()) < categorize_threshhold]
    facters_pairs = [facters_pair for facters_pair in combinations(categorize_facters_list, 2)]
    separate_facter = []
    print(facters_pairs)
    for facters_pair in facters_pairs:
        cross_tab = pd.crosstab(df[facters_pair[0]], df[facters_pair[1]])
        print(cross_tab)
        p = stats.chi2_contingency(cross_tab, correction=False)[1] #correction:イェーツの補正
        print(p)
        if p < 0.05 :
            separate_facter.append(facters_pair)
    return separate_facter#[[a,a],[b,b]]


def fisher_exact_test(df, categorize_threshhold = 3, facters_list = []):
    categorize_facters_list = [facter for facter in facters_list if len(df[facter].unique()) < categorize_threshhold]
    facters_pairs = [facters_pair for facters_pair in combinations(categorize_facters_list, 2)]
    separate_facter = []
    for facters_pair in facters_pairs:
        cross_tab = pd.crosstab(df[facters_pair[0]], df[facters_pair[1]])
        p = stats.fisher_exact(cross_tab)[1] #correction:イェーツの補正
        if p < 0.05 :
            separate_facter.append(facters_pair)
    return separate_facter#[[a,a],[b,b]]


In [ ]:
cross_tab = pd.crosstab(df[facters_list[0]], df[facters_list[1]])
cross_tab

In [ ]:
column_sums = cross_tab.sum().to_list()        #c   
row_sums = cross_tab.sum(axis = 1).to_list()   #r
full_sum = sum(column_sums)                    #n

In [ ]:
import math
import itertools
def list_factorial_sum(taraget_list):
    sum = 0
    for target in taraget_list:
        m = math.factorial(target)
        sum += m
    return sum


column_factorial = 0
for column_sum in column_sums:
    m = math.factorial(column_sum)
    column_factorial += m

row_factorial = 0
for row_sum in row_sums:
    m = math.factorial(row_sum)
    row_factorial += m

n_factorial = math.factorial(full_sum)

values_factorial = 0
cross_tab_values = list(itertools.chain.from_iterable(cross_tab.values.tolist()))
for cross_tab_value in cross_tab_values:
    m = math.factorial(cross_tab_value)
    values_factorial += m

    

In [ ]:
cross_tab_values = cross_tab.values.tolist()
for cross_tab_value in cross_tab_values:    #行ごと
    rows_value_factorial_sum = 0
    for i, cross_tab_value in enumerate(cross_tab_values):
        row_value_factorial_sum = list_factorial_sum(cross_tab_value)
        row_factorial = math.factorial(column_sums[i])
        k = row_value_factorial_sum/row_factorial
        rows_value_factorial_sum += k
        

In [ ]:
rows_value_factorial_sum


In [ ]:
arc_p = n_factorial/column_factorial
p

In [ ]:
arc_p = n_factorial/column_factorial
p
rows_value_factorial_sum


In [ ]:
cross_tab_values = cross_tab.values.tolist()
for cross_tab_value in cross_tab_values:    #行ごと
    rows_value_factorial_sum = 0
    

# 綺麗なエクセルを整形する

In [ ]:
import openpyxl

book = openpyxl.Workbook()

In [ ]:
df = df

In [ ]:
df['haba'] = f"{df['random_num1']}" + f"{df['random_num2']}"

In [ ]:
df